# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [70]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
import dask.dataframe as dd

/var/folders/kb/810cbr5s4ng1rrn3k78ct01h0000gn/T/ipykernel_81362/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [41]:
import os
from glob import glob

# Write your code below.

# loading environmental variable Price_data
price_data = os.getenv("PRICE_DATA") 

# displaying path of all parquet files
parquet_files = glob(os.path.join(price_data, "*/*.parquet/part.0.parquet"))
parquet_files


['../../05_src/data/prices/WELL/WELL_2021.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2007.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2017.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2016.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2006.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2020.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2004.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2014.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2022.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2023.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2015.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2005.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2000.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2010.parquet/part.0.parquet',
 '../../05_src/data/prices/WELL/WELL_2019.parquet/part.0.parqu

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [64]:
# Write your code below.

import dask.dataframe as dd
import numpy as np 

# loading parquet files into dask dataframe
ddf = dd.read_parquet(parquet_files)

dd_feat = (ddf.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1), Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low']
)))

/var/folders/kb/810cbr5s4ng1rrn3k78ct01h0000gn/T/ipykernel_81362/3007125981.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (ddf.groupby('ticker', group_keys=False).apply(


In [67]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
WELL,2024-01-02,90.309998,90.629997,89.790001,90.239998,88.595901,1906300,Real Estate,Health Care REITs,2024,NaN,NaN,NaN,0.839996
WELL,2024-01-03,89.800003,90.290001,88.760002,88.910004,87.290138,2079200,Real Estate,Health Care REITs,2024,90.239998,88.595901,-0.014738,1.529999
WELL,2024-01-04,88.940002,89.459999,88.309998,88.699997,87.083954,1824900,Real Estate,Health Care REITs,2024,88.910004,87.290138,-0.002362,1.150002
WELL,2024-01-05,88.830002,89.360001,87.900002,88.519997,86.907234,3072400,Real Estate,Health Care REITs,2024,88.699997,87.083954,-0.002029,1.459999
WELL,2024-01-08,88.489998,89.820000,88.300003,89.809998,88.173729,1787200,Real Estate,Health Care REITs,2024,88.519997,86.907234,0.014573,1.519997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMP,2006-12-22,54.669998,54.880001,54.299999,54.650002,37.633648,469200,Financials,Asset Management & Custody Banks,2006,54.709999,37.674976,-0.001097,0.580002
AMP,2006-12-26,54.689999,54.930000,54.590000,54.830002,37.757607,603700,Financials,Asset Management & Custody Banks,2006,54.650002,37.633648,0.003294,0.340000
AMP,2006-12-27,54.830002,55.270000,54.610001,54.820000,37.750713,724200,Financials,Asset Management & Custody Banks,2006,54.830002,37.757607,-0.000183,0.660000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [69]:
# Write your code below.
# converting to pandas dataframe
pdf = dd_feat.compute()

# calculating rolling average return
pdf['rolling average return'] = pdf['returns'].rolling(10).mean()
pdf.head()


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,rolling average return
ticker,,,,,,,,,,,,,,,
WELL,2024-01-02,90.309998,90.629997,89.790001,90.239998,88.595901,1906300,Real Estate,Health Care REITs,2024,NaN,NaN,NaN,0.839996,NaN
WELL,2024-01-03,89.800003,90.290001,88.760002,88.910004,87.290138,2079200,Real Estate,Health Care REITs,2024,90.239998,88.595901,-0.014738,1.529999,NaN
WELL,2024-01-04,88.940002,89.459999,88.309998,88.699997,87.083954,1824900,Real Estate,Health Care REITs,2024,88.910004,87.290138,-0.002362,1.150002,NaN
WELL,2024-01-05,88.830002,89.360001,87.900002,88.519997,86.907234,3072400,Real Estate,Health Care REITs,2024,88.699997,87.083954,-0.002029,1.459999,NaN
WELL,2024-01-08,88.489998,89.820000,88.300003,89.809998,88.173729,1787200,Real Estate,Health Care REITs,2024,88.519997,86.907234,0.014573,1.519997,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to convert to pandas data frame. We can calculate both in dask and pandas but it would be better to do in dask data frame beacause dask manage the memory by processing large data sets in chunks.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.